<a href="https://colab.research.google.com/github/JuniorHZ19/ML/blob/main/perceptronmulticapa3.0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision

In [ ]:
import torch
from torch.utils.data import Dataset
import pandas as pd
import sklearn
from sklearn.preprocessing import StandardScaler



class MiDataSet(Dataset):

  def __init__(self,csv_file):

     data=pd.read_csv(csv_file)

     self.x,self.y= self.limpiezaDatos(data)

     self.samples=self.y.shape[0]


  def __getitem__(self,id):

   return self.x[id],self.y[id]


  def __len__(self):
   return self.samples


  def limpiezaDatos(self,data):

    data=data.dropna(how="all")

    data=data.drop_duplicates()

    data=data.drop(columns=["name","ticket","cabin","boat","body","home.dest"])

    data[["age","fare"]]=data[["age","fare"]].fillna(data[["age","fare"]].mean())

    data["embarked"]=data["embarked"].fillna("S")

    data["age"]=data["age"].astype(int)

    data=pd.get_dummies (data,columns=["sex","embarked"],prefix=["sex","embarked"])

    variables_x=data.drop(columns=["survived"])

    print(variables_x.head())

    print(data["survived"].head())
    self.scaler=StandardScaler()

    variables_x_esclados=self.scaler.fit_transform(variables_x)

    variable_y=data["survived"].values


    datos_x=torch.from_numpy(variables_x_esclados).float()

    datos_y=torch.from_numpy(variable_y).float()



    return datos_x,  datos_y







In [ ]:
import sklearn

from sklearn.model_selection import train_test_split

mis_datos=MiDataSet("/content/titanic.csv")



X_entrenamiento, X_prueba, y_entrenamiento, y_prueba = train_test_split(mis_datos.x, mis_datos.y, test_size=0.2, random_state=0)

print(X_entrenamiento.shape)

print(X_prueba.shape)

print(y_entrenamiento.shape)

print(y_prueba.shape)




In [ ]:



from torch.utils.data import DataLoader, TensorDataset


conjunto_entrenamiento = TensorDataset(X_entrenamiento, y_entrenamiento)

conjunto_prueba = TensorDataset(X_prueba, y_prueba)



data_por_lote_entrenamiento=DataLoader(conjunto_entrenamiento,batch_size=100,shuffle=True)

data_por_por_lote_test=DataLoader(conjunto_prueba,batch_size=100,shuffle=True)



datos_features_train,datos_label_train=next(iter(data_por_lote_entrenamiento))

datos_features_test,datos_label_test=next(iter(data_por_por_lote_test))


print(datos_features_test.shape)




torch.Size([100, 10])


In [ ]:
import torch.nn as nn
import torch.nn.functional as F

modelo=torch.Sequential(

                        torch.nn.Linear(input_size,3),
                        torch.nn.ReLU(),

                        torch.nn.Linear(3,4),
                        torch.nn.ReLU(),

                        torch.nn.Linear(4,1),
                        torch.nn.Sigmoid()
)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F



class PerceptronMulticapa(nn.Module):

   def __init__(self,input_size):
    super(PerceptronMulticapa,self).__init__()

    self.capa_oculta1=nn.Linear(input_size,1000)

    self.capa_oculta2=nn.Linear(1000,500)

    self.capa_oculta3=nn.Linear(500,200)

    self.capa_salida =nn.Linear(200,1)

   def forward(self,x):

     x=self.capa_oculta1(x)
     x=F.relu(x)

     x=self.capa_oculta2(x)
     x=F.relu(x)

     x=self.capa_oculta3(x)
     x=F.relu(x)

     x=self.capa_salida(x)
     output=torch.sigmoid(x)


     return output


In [ ]:

if torch.cuda.is_available():
    print("CUDA está disponible.")
    device = torch.device("cuda")
else:
    print("CUDA no está disponible. Se utilizará la CPU.")
    device = torch.device("cpu")

In [ ]:



#funcion clasificador:
def clasificador(y_pred_test):

  umbral = 0.6
  y_pred_test_binario = (y_pred_test >= umbral).to(torch.float32)

  return y_pred_test_binario

#funcion acutity:
def accurity(y_test,y_real):
   correctos=0
   for predicho, real in zip(y_test, y_real):
    if predicho==real:
     correctos+=1
   return correctos/len(y_test)


import torch.optim as optim

input_size=X_entrenamiento.shape[1]

PerceptronMulti=PerceptronMulticapa(input_size)

criterio=nn.BCELoss()

optimizer=optim.SGD(PerceptronMulti.parameters(),lr=0.1)

epochs=500

def TrainMLP(modelo,epochs,optimisador,criterio,input_size):

 for epoch in range(epochs):

  model.train()
  X_entrenamiento=X_entrenamiento.to(device)
  y_predict=PerceptronMulti(X_entrenamiento)

  loss=criterio(y_predict,y_entrenamiento.view(-1,1))

  loss.backward()

  optimizer.step()

  optimizer.zero_grad()
  modelo.eval()
  with torch.no_grad():
   X_prueba=X_prueba.to(device)
   y_predict_test=PerceptronMulti(X_prueba)
   y_predic_test_clasificado=clasificador(y_predict_test)
   Precision=accurity(y_prueba,y_predic_test_clasificado)



  print(f'Epoca[{epoch+1}],Perdida: {loss.item():.4f} ,Accuracy: {Precision:.4f}')


#nuevo datos:

def escalarNuevaData(data):

    scaler=StandardScaler()

    datos_escalados=scaler.fit_transform(data)

    datos_x_tensor=torch.from_numpy(datos_escalados).float()

    return datos_x_tensor

nuevo_x=escalarNuevaData([[1,29,0,0,211,1,0,0,0,1]])

y_predict=PerceptronMulti(nuevo_x)

print(clasificador(y_predict))

In [ ]:


y_pred_multiclases = torch.tensor([[0.1, 1.8, 0.8, 0.4, 2],
                                   [0.5, 2, 3.5, 5, 2.1]])

def clasificador(y_pred_test):

   max_values_tensor = torch.empty((0,), dtype=torch.float32)

   for tensor in y_pred_test:

     max_values, max_indices  = torch.max(tensor, dim=0)

     max_values_tensor = torch.cat((max_values_tensor, max_indices.unsqueeze(0)))

   return max_values_tensor

print(clasificador(y_pred_multiclases))


tensor([4., 3.])
